# This script will archive key information about your Tableau Workbooks to Dynamo

+ Name
+ Archival Time
+ Expiration Time
+ Workbook ID
+ Owner ID
+ Site ID
+ Permissions

Once this information is in Dynamo, it can be retrieved quickly like so
Access permissions with the following
```
response = dynamo.get_item(
     TableName =  'Tableau_Workbooks',
     Key = {
         'Name': {
             'S': 'Workbook Name'
         }
     }
   
 )
```

First step is to get all the packages you need. This script doesn't use the Server Client Library because it doesn't have a method for permissions.

In [ ]:
#Import your packages
from slacker import Slacker
import psycopg2
import boto3
import time
import datetime
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
import json
import multiprocessing as mp
from datetime import datetime

In order to speed things up, we will be using the multiprocessing library. This allows Python to work on all your cores, in parallel, so you don't have to iterate over everythign one at a time. How much time does this save? The initial version of this script took >20m. Now it runs in <5.

In [ ]:
#Set your credentials and other settings
username = ''
password = ''
serverurl = ''
current_time = time.time()
slack = Slacker('')
#This will cause the record to expire in 180 days
#We rewrite every N days, so this will always get reset, unless content is deleted
expiry = current_time + 15552000
cpus=mp.cpu_count()
dynamo = boto3.client('dynamodb')

We are going to use a special function in Dynamo called TTL (time-to-live). We archive all our workbooks regularly. If a workbook gets screened out due to inactivity, we want to make sure we can re-publish in its prior state. We record every workbook in Server in Dynamo, and add an expiration key of time + 180 days.

This means that if a workbook gets deleted, it won't be updated and has 180 days to be republished. After that, the record expires out of Dynamo **AUTOMATICALLY**

---

Next up, we define our functions. These are the raw Python functions for accessing the Server REST API. We also use Boto3, the official python client for AWS. We set up a Dynamo interface and create our *create item* and *update item* functions

> Note that if you are running this on a machine without AWS credentials, this won't work. You either need to pre-configure them, or run them on a machine with an appropriate IAM role.

In [ ]:
#Define your functions
def toDynamo(item):
    dynamo.put_item(
        TableName='Tableau_Workbooks',
        Item={
            'Name': {'S':item['name']},
            'ID': {'S': item['ID']},
            'Owner': {'S': item['owner']},
            'Site': {'S':item['site']},
            'Write': {'N': str(current_time)},
            'Expiry': {'N': str(expiry)},            
        }
    )

def updateDynamo(name,update):
    dynamo.update_item(
        TableName='Tableau_Workbooks',
        Key={
            'Name': {'S':name},
        },
        UpdateExpression='SET Perms = :r',
        ExpressionAttributeValues={
            ':r': {'S':update}
        }
    )

def login(site_name):
    url = serverurl+"/api/2.6/auth/signin"
    payload = "{\n  \"credentials\": {\n    \"name\": \""+username+"\",\n    \"password\": \""+password+"\",\n    \"site\": {\n      \"contentUrl\": \""+site_name+"\"\n    }\n  }\n}"
    headers = {
        'content-type': "application/json",
        'accept': "application/json",
        }
    response = requests.request("POST", url, data=payload, headers=headers, verify=False)
    token = response.json()
    return token.values()[0]['token']

def get_sites(auth):
    url = serverurl+"/api/2.6/sites"
    headers = {
        'content-type': 'application/json',
        'accept': 'application/json',
        'X-Tableau-Auth': auth
    }
    response = requests.request('get', url=url, headers=headers)
    json = response.json()
    response_json = json['sites']['site']
    sites = []
    for response in response_json:
        values = [response['contentUrl'], response['id']]
        site = {'site_id':values[1], 'url':values[0]}
        sites.append(site)
    return sites

def get_workbooks(site_id,auth):
    url = serverurl+"/api/2.6/sites/"+site_id+"/workbooks?pageSize=1000"
    headers = {
        'content-type': 'application/json',
        'accept': 'application/json',
        'X-Tableau-Auth': auth
    }
    response = requests.request('get', url=url, headers=headers)
    json = response.json()
    return json

def perms(data):
    url = serverurl+"/api/2.6/sites/"+data['siteid']+"/workbooks/"+data['workbook']+'/permissions'
    headers = {
        'content-type': "application/json",
        'accept': "application/json",
        'X-Tableau-Auth': data['auth']
        }
    response = requests.request("GET", url, headers=headers)
    updateDynamo(data['name'], response.text)
    
pool = mp.Pool(processes=cpus)

Here at the bottom we create our pool - which is linked to the number of cores we have on our machine. Then we simply execute our functions. When we get to a point where we have all the information we need for our Dynamo update, we add them to the pool and let the parallelism commence.

In [ ]:
#Go
auth = login('')
sites = get_sites(auth)

In [ ]:
workbooks_list = []
for site in sites:
    auth = login(site['url'])
    workbooks = get_workbooks(site['site_id'],auth)
    if len(workbooks['workbooks'])==0:
        print('No workbooks to archive')
    else: 
        workbook_filter = workbooks['workbooks']['workbook']        
        for workbook in workbook_filter:
            record = {'name':workbook['name'],'ID':workbook['id'],'owner':workbook['owner']['id'],'site':site['site_id']}
            workbooks_list.append(record)
pool.map(toDynamo,workbooks_list) 

In [ ]:
workbook_ids = []
for site in sites:        
    site_id = site['site_id']
    auth = login(site['url'])
    workbooks = get_workbooks(site_id,auth)
    if len(workbooks['workbooks'])==0:
        print('No workbooks to archive')
    else: 
        workbook_filter = workbooks['workbooks']['workbook']  
        for workbook in workbook_filter:
            record = {'siteid':site_id,'workbook':workbook['id'],'auth':auth, 'name':workbook['name']}
            workbook_ids.append(record)
pool.map(perms,workbook_ids)

Now that everything is complete, we close down the pool and send a notification that the job is complete.

In [ ]:
#Cleanup
pool.close() 
pool.join() 
slack.chat.post_message('@christ', 'Workbook details archived')